                                                          LiDAR Point Cloud Vectorization

LiDAR Point Cloud Vectorization converts raw LiDAR data into structured vector information.

It involves processing 3D point cloud data captured by LiDAR sensors.

The process includes noise removal, classification, and feature extraction.

Algorithms segment and identify objects like buildings, vegetation, and roads.

Vectorization transforms point clouds into polygons, lines, and points.


In [9]:
# Base Libraries
import numpy as np  # Importing NumPy for numerical operations
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting
import pandas as pd  # Importing Pandas for data manipulation
import tkinter as tk  # Importing Tkinter for GUI components
from tkinter import filedialog  # Importing filedialog submodule from Tkinter

# 3D Libraries
# import open3d as o3d  # Importing Open3D for 3D visualization and processing
import laspy  # Importing laspy for reading and writing LAS file format
print(laspy.__version__)  # Printing laspy version

# Geospatial Libraries
import rasterio  # Importing rasterio for reading and writing raster data
import alphashape as ash  # Importing alphashape for creating alpha shapes
import geopandas as gpd  # Importing geopandas for geospatial data manipulation
import shapely as sh  # Importing shapely for geometric operations

from rasterio.transform import from_origin  # Importing from_origin from rasterio.transform
from rasterio.enums import Resampling  # Importing Resampling enumeration from rasterio.enums
from rasterio.features import shapes   # Importing shapes function from rasterio.features
from shapely.geometry import Polygon    # Importing Polygon class from shapely.geometry



2.5.3


In [10]:
las = laspy.read('points.laz')

# Exploring the classification areas
print(np.unique(las.classification))

# Print dimensions' names
print([dimension.name for dimension in las.point_format.dimensions])

# Check available attributes for the VLR object
vlr_attributes = dir(las.vlrs[2])

# Print available attributes for the VLR object
print("VLR Attributes:", vlr_attributes)

# Use the correct attribute based on your requirements
# For example, if you want to get the raw data as a string, you might use:
if 'raw_data' in vlr_attributes:
    crs = las.vlrs[2].raw_data
    print("CRS Raw Data:", crs)
else:
    print("VLR object does not have 'raw_data' attribute.")

LaspyException: No LazBackend selected, cannot decompress data

In [ ]:
# Data preprocessing 

#mask to filter points
pts_mask = las.classification == 1


#coordinates
xyz_t = np.vstack((las.x[pts_mask],las.y[pts_mask],las.z[pts_mask]))

#transformation and visualization
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points =o3d.utility.Vector3dVector(xyz_t.transpose())

#translation of point cloud 
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(pcd_center)

#visualization of result
o3d.visualization.draw_geometries([pcd_o3d])



In [ ]:
# Data preprocessing  for ground points

#mask to filter points
pts_mask = las.classification == 2


#coordinates
xyz_t = np.vstack((las.x[pts_mask],las.y[pts_mask],las.z[pts_mask]))

#transformation and visualization
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points =o3d.utility.Vector3dVector(xyz_t.transpose())

#translation of point cloud 
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(pcd_center)

#visualization of result
o3d.visualization.draw_geometries([pcd_o3d])



In [ ]:
# Mask to filter building points (classification 1)
building_mask = las.classification == 1

# Mask to filter ground points (classification 2)
ground_mask = las.classification == 2

# Coordinates of building points
xyz_building = np.vstack((las.x[building_mask], las.y[building_mask], las.z[building_mask]))

# Coordinates of ground points
xyz_ground = np.vstack((las.x[ground_mask], las.y[ground_mask], las.z[ground_mask]))

# Downsample the data
building_downsampled = xyz_building[:, ::10]  # Adjust the step size as needed
ground_downsampled = xyz_ground[:, ::10]

# Calculate distances for the downsampled data
distances = np.linalg.norm(building_downsampled[:, np.newaxis, :] - ground_downsampled[:, :, np.newaxis], axis=0)

# Filter out invalid distances (e.g., distances between the same points)
valid_distances = distances[~np.isnan(distances)]

# Calculate the average distance
avg_distance = np.mean(valid_distances)

print(f"Average distance between ground and building points: {avg_distance} units")

In [ ]:
# Unsupervised clustering using DBSCAN

epsilon = 2  # Set the epsilon parameter for DBSCAN
min_cluster_points = 100  # Set the minimum number of points in a cluster

# Perform DBSCAN clustering on the point cloud
labels = np.array(pcd_o3d.cluster_dbscan(eps=epsilon, min_points=min_cluster_points))
max_label = labels.max()  # Get the maximum label assigned by DBSCAN
print(f"Point cloud has {max_label+1} clusters")

# Color pattern to generate visualization
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))  # Generate colors based on cluster labels
colors[labels < 0] = 0  # Set colors for noise points to black
pcd_o3d.colors = o3d.utility.Vector3dVector(colors[:, :3])  # Set colors to the point cloud

# Point Cloud Visualization
o3d.visualization.draw_geometries([pcd_o3d])  # Visualize the clustered point cloud


In [ ]:
#selection of a particular segment
sel = 1
segment = pcd_o3d.select_by_index(np.where(labels == sel)[0])
o3d.visualization.draw_geometries([segment])

In [ ]:
# Extracting the outline of the above selection

points_2D = np.asarray(segment.points)[:, 0:2]  # Extracting only x and y coordinates from the segment points

# Alpha shape 

# Generate an alpha shape from the 2D points with a specified alpha value
building_vector = ash.alphashape(points_2D, alpha=0.5)

In [ ]:
#storing in geodataframe
building_gdf = gpd.GeoDataFrame(geometry=[building_vector],crs='EPSG:26910')
building_gdf.head(1)

In [ ]:

points_1D = np.asarray(segment.points)[:, 2]  # Extracting only the z coordinates from the segment points

# Print the local minima and maxima along the z-axis
print("The local minima along the z-axis:", np.min(points_1D))
print("The local maxima along the z-axis:", np.max(points_1D))

# Create a histogram of the z-axis coordinates
plt.hist(points_1D, bins='auto')  # 'auto' option automatically determines the number of bins
plt.title("Histogram of above")  # Set the title of the histogram
plt.show()  # Display the histogram


In [ ]:
# 2D to 3D Library

# base layer
vertices = list(building_vector.exterior.coords)

# construct in open3d
Polygon_2d = o3d.geometry.LineSet()
Polygon_2d.points = o3d.utility.Vector3dVector([point + (0,) for point in vertices])
Polygon_2d.lines = o3d.utility.Vector2iVector([(i, (i + 1) % len(vertices)) for i in range(len(vertices))])

# show
o3d.visualization.draw_geometries([Polygon_2d])


In [ ]:
#genrate some element for the extruded

extrusion = o3d.geometry.LineSet()

Polygon_2d = o3d.geometry.LineSet()
extrusion.points = o3d.utility.Vector3dVector([point + (0,) for point in vertices])
extrusion.lines = o3d.utility.Vector2iVector([(i, (i + 1) % len(vertices)) for i in range(len(vertices))])
o3d.visualization.draw_geometries([Polygon_2d,extrusion])

#plot the vertices

temp = Polygon_2d + extrusion
temp.points
temp_o3d = o3d.geometry.PointCloud()
temp_o3d.points = temp.points
o3d.visualization.draw_geometries([temp_o3d])


In [ ]:
import open3d as o3d
import numpy as np

# Create a simple mesh (replace this with your actual mesh creation logic)
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
triangles = np.array([[0, 1, 2]])
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)

# Translation vector
pcd_center = [1, 2, 3]

# Translate the mesh
mesh.translate(pcd_center)

# Now, you can proceed with writing the mesh to a file using write_triangle_mesh
output_mesh_path = r'D:\Resume Project\Lidar_cloud_data\house_sample.ply'
o3d.io.write_triangle_mesh(output_mesh_path, mesh, write_ascii=False, compressed=True, print_progress=False)
